<a href="https://colab.research.google.com/github/ryotaiizuka2027-eng/tex/blob/main/Portfolio_Analysis_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np

def calculate_top_quantile_metrics(
    df,
    score_col='score',
    date_col='date',
    dateym_col='dateym',
    ticker_col='ticker',
    close_col='close',
    n_quantiles=5,
    start_ym=None,
    end_ym=None
):
    """
    すでに用意されたdfを用いて、スコア上位層のポートフォリオパフォーマンスを計算する関数。

    Parameters:
    -----------
    df : pd.DataFrame
        入力データフレーム（月次データ）。以下の列が必要:
        - date_col (datetime)
        - dateym_col (int/str, e.g., 202101)
        - ticker_col (str)
        - close_col (float)
        - score_col (float)
    score_col : str
        スコアが入っている列名
    n_quantiles : int
        分位数（例: 5なら上位20%）
    """

    # 元データを破壊しないようにコピー
    work_df = df.copy()

    # 1. データのソート（リターン計算のため）
    work_df = work_df.sort_values([ticker_col, date_col])

    # 2. 個別銘柄のリターン計算
    # Simple Return (P_t / P_{t-1} - 1)
    # ※ close列が既にリターンになっている場合はこの行をコメントアウトし、
    #    close_colをリターン列として扱ってください
    work_df['return'] = work_df.groupby(ticker_col)[close_col].pct_change()

    # リターン計算できない行（各銘柄の初月）を除去
    work_df = work_df.dropna(subset=['return'])

    # 3. 期間フィルタリング (dateymを使用)
    if start_ym is not None:
        work_df = work_df[work_df[dateym_col].astype(int) >= int(start_ym)]
    if end_ym is not None:
        work_df = work_df[work_df[dateym_col].astype(int) <= int(end_ym)]

    if work_df.empty:
        print("Warning: 指定された期間のデータが存在しません。空のDataFrameを返します。")
        return pd.DataFrame(), pd.DataFrame()

    # 4. 月ごとに分位（Quantile）を計算し、トップ層を抽出する関数
    def get_top_layer_returns(group):
        # スコア有効データのみ
        valid_group = group.dropna(subset=[score_col])

        # データ点数が少なすぎて分位計算できない場合
        if len(valid_group) < n_quantiles:
            return np.nan

        try:
            # qcutでグループ分け (0 ~ n_quantiles-1)
            # method='first'で同値スコアの順位付け問題を回避
            valid_group['quantile'] = pd.qcut(
                valid_group[score_col].rank(method='first'),
                q=n_quantiles,
                labels=False
            )
        except ValueError:
            return np.nan

        # 一番スコアが高い層 (Top Quantile: n_quantiles - 1) を抽出
        top_layer = valid_group[valid_group['quantile'] == (n_quantiles - 1)]

        # 等ウェイト(Equal Weight)でリターンを平均
        return top_layer['return'].mean()

    # 各月（date_col）ごとに適用
    # 日付インデックスを持つSeriesが返ります
    portfolio_returns = work_df.groupby(date_col).apply(get_top_layer_returns)
    portfolio_returns = portfolio_returns.dropna()

    # 5. 各種指標の計算
    if portfolio_returns.empty:
        print("Warning: ポートフォリオのリターンが計算できませんでした（データ不足など）。")
        return pd.DataFrame(), pd.DataFrame()

    # (1) 累積リターン (Geometric Cumulative Return)
    cum_return = (1 + portfolio_returns).prod() - 1

    # (2) 累和リターン (Arithmetic Sum Return)
    sum_return = portfolio_returns.sum()

    # (3) リスク (年率標準偏差)
    # 月次データ前提のため sqrt(12) を乗算
    risk_annualized = portfolio_returns.std() * np.sqrt(12)

    # (4) リターン/リスク (Risk-Return Ratio)
    # 年率平均リターン / 年率リスク
    mean_return_annualized = portfolio_returns.mean() * 12
    return_risk_ratio = mean_return_annualized / risk_annualized if risk_annualized != 0 else 0

    # 6. 結果のデータフレーム化
    metrics_data = {
        '指標': [
            '累積リターン (Cumulative)',
            '累和リターン (Sum)',
            'リスク (年率標準偏差)',
            'リターン/リスク (Risk-Return Ratio)'
        ],
        '値': [
            cum_return,
            sum_return,
            risk_annualized,
            return_risk_ratio
        ]
    }
    metrics_df = pd.DataFrame(metrics_data)

    # 月次推移データ（グラフ描画用）
    monthly_perf_df = portfolio_returns.to_frame(name='portfolio_return')
    monthly_perf_df['cumulative_return'] = (1 + monthly_perf_df['portfolio_return']).cumprod() - 1

    return metrics_df, monthly_perf_df

# ==========================================
# 実行部分 (dfは既に存在すると想定)
# ==========================================

# 以下の変数設定を変更して使用してください
# ------------------------------------------------
USER_SCORE_COL = 'score'      # 分析に使うスコア列の名前
USER_START_YM  = 201501       # 開始年月 (YYYYMM)
USER_END_YM    = 202312       # 終了年月 (YYYYMM)
USER_QUANTILES = 5            # 分位数 (5=上位20%, 10=上位10%)
# ------------------------------------------------

# 変数 `df` が存在するかチェックしてから実行
if 'df' in locals() and isinstance(df, pd.DataFrame):
    print(f"--- Calculating Metrics for '{USER_SCORE_COL}' (Top {100/USER_QUANTILES:.0f}%) ---")

    metrics_df, history_df = calculate_top_quantile_metrics(
        df,
        score_col=USER_SCORE_COL,
        date_col='date',      # df内の日付列名
        dateym_col='dateym',  # df内の年月列名
        ticker_col='ticker',  # df内のティッカー列名
        close_col='close',    # df内の終値列名
        n_quantiles=USER_QUANTILES,
        start_ym=USER_START_YM,
        end_ym=USER_END_YM
    )

    if not metrics_df.empty:
        print("\n[Performance Metrics]")
        print(metrics_df)

        # 必要であればCSV出力など
        # metrics_df.to_csv('portfolio_metrics.csv', index=False)
    else:
        print("結果が得られませんでした。")

else:
    print("エラー: 変数 'df' が見つかりません。")
    print("このコードを実行する前に、データを読み込んだ 'df' を用意してください。")